# GOAL: Improve the 2|0 calls before running k-means clustering

## 0. Get dataframes

### 0a. Load in data

In [1]:
source("utils_high-depth.R")
library(GenomicRanges)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.table::transpose()
✖ lubridate::wday() 

In [2]:
#cells_to_use <- Step1_filter_to_qc_pass_cells()

In [3]:
# ----------------------------
# SET FIXED PARAMETERS
# ----------------------------
prefix <- "/n/data1/hms/dbmi/park/ann_caplin/SMaHT/PTA"
batch_names <- c("batch-6-uniform_lung", "batch-7-uniform_colon")
binsize <- "500000"
lambda_str <- "1024-512"
lambda <- 512
hiscanner_cell_type_ix=3

In [4]:
seg_merged_df <- as.data.frame(get_cell_data(prefix, batch_names,
                                             binsize, lambda_str, 
                                             lambda, file_suffix="_seg_merged",
                                            hiscanner_dir_name="hiscanner_dsa_multisample-false"))

/n/data1/hms/dbmi/park/ann_caplin/SMaHT/PTA/batch-6-uniform_lung/hiscanner_dsa_multisample-false/binsize_500000_lambda_1024-512

Found 56 input_table.txt files

batch-6-uniform_lung

Reading cell_name.txt files...

Loaded data for 4536 rows

/n/data1/hms/dbmi/park/ann_caplin/SMaHT/PTA/batch-7-uniform_colon/hiscanner_dsa_multisample-false/binsize_500000_lambda_1024-512

Found 46 input_table.txt files

batch-7-uniform_colon

Reading cell_name.txt files...

Loaded data for 1737 rows



In [5]:
orig_seg_merged_df_colnames <- colnames(seg_merged_df)

In [6]:
seg_merged_df$CN <- as.character(seg_merged_df$CN)

# Split CN into two numeric components
cn_split <- str_split(seg_merged_df$CN, "\\|")

# Extract alleles as numeric values
seg_merged_df$CN_A <- as.numeric(sapply(cn_split, `[`, 1))
seg_merged_df$CN_B <- as.numeric(sapply(cn_split, `[`, 2))

# Compute total copy number
seg_merged_df$CN_total <- seg_merged_df$CN_A + seg_merged_df$CN_B

seg_merged_df$length <- seg_merged_df$end - seg_merged_df$start
seg_merged_df$amt_dna <- seg_merged_df$length*seg_merged_df$CN_total
seg_merged_df$is_CNV <- (seg_merged_df$CN_A != 1 | seg_merged_df$CN_B != 1)
seg_merged_df$is_LOH <- (seg_merged_df$CN_A == 0 | seg_merged_df$CN_B == 0)

In [7]:
cell_df <- get_cell_data(prefix, batch_names, binsize, lambda_str, lambda, file_suffix="",
                        hiscanner_dir_name="hiscanner_dsa_multisample-false")

/n/data1/hms/dbmi/park/ann_caplin/SMaHT/PTA/batch-6-uniform_lung/hiscanner_dsa_multisample-false/binsize_500000_lambda_1024-512

Found 56 input_table.txt files

batch-6-uniform_lung

Reading cell_name.txt files...

Loaded data for 281294 rows

/n/data1/hms/dbmi/park/ann_caplin/SMaHT/PTA/batch-7-uniform_colon/hiscanner_dsa_multisample-false/binsize_500000_lambda_1024-512

Found 46 input_table.txt files

batch-7-uniform_colon

Reading cell_name.txt files...

Loaded data for 229868 rows



In [8]:
orig_cell_df_colnames <- colnames(cell_df)

In [9]:
cell_df$CN_signal <- cell_df$RDR * cell_df$gamma

### 0b. Filter to autosomes

In [10]:
cell_df <- cell_df %>% filter(CHROM %in% c(1:22))
seg_merged_df <- seg_merged_df %>% filter(chrom %in% c(1:22))

In [11]:
# DO NOT filter to QC pass cells (yet!) -- apply different QC to cells
#cell_df <- cell_df %>% filter(cell_name %in% cells_to_use)
#seg_merged_df <- seg_merged_df %>%  filter(cell_name %in% cells_to_use)

## 2. Perform k-bin BAF smoothing to find signal for CN-LOH

### 2c. Implementation

In [12]:
library(dplyr)
library(data.table)

k <- 10
baf_cutoff <- 0.25

# Step 1: Bin diploid regions only
binned_df <- cell_df %>%
  filter(CN_total == 2, !is.na(BAF)) %>%
  arrange(cell_name, CHROM, START) %>%
  group_by(cell_name, CHROM) %>%
  mutate(bin_idx = row_number(),
         bin_group = ceiling(bin_idx / k)) %>%
  group_by(cell_name, CHROM, bin_group) %>%
  summarise(
    start = min(START),
    end = max(END),
    mean_baf = mean(BAF, na.rm = TRUE),
    .groups = "drop"
  )

binned_df <- binned_df %>%
  mutate(
    cnloh_candidate = mean_baf < baf_cutoff,
    seg_id = data.table::rleid(cnloh_candidate)
  )

loh_segments <- binned_df %>%
  filter(cnloh_candidate) %>%
  group_by(cell_name, CHROM, seg_id) %>%
  summarise(
    start = min(start),
    end = max(end),
    n_bins = k*n(),
    mean_baf = mean(mean_baf),
    CN_total = 2,
    CN = "2|0",
    .groups = "drop"
  )


In [13]:
loh_segments

cell_name,CHROM,seg_id,start,end,n_bins,mean_baf,CN_total,CN
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>
SMACS2T3VKE6,7,2,10005,31358909,60,0.06999303,2,2|0
SMACS3WFHOU9,1,4,147811061,170529950,40,0.22668178,2,2|0
SMACS3WFHOU9,1,6,196341876,201558736,10,0.22657325,2,2|0
SMACS3WFHOU9,1,8,217315709,238202584,40,0.23617147,2,2|0
SMACS3WFHOU9,12,10,97059233,102200367,10,0.19382907,2,2|0
SMACS3WFHOU9,12,12,107363689,112433752,10,0.24725438,2,2|0
SMACS3WFHOU9,14,14,61462910,66603616,10,0.20734752,2,2|0
SMACS3WFHOU9,14,16,92172118,106883619,30,0.17369100,2,2|0
SMACS3WFHOU9,16,18,53579665,63863913,20,0.21555301,2,2|0


## 3. Integrate LOH calls into seg_merged_df and cell_df

In [14]:
library(dplyr)
library(GenomicRanges)
library(data.table)

# Ensure consistent column types
cell_df$CHROM <- as.character(cell_df$CHROM)
seg_merged_df$chrom <- as.character(seg_merged_df$chrom)
loh_segments$CHROM <- as.character(loh_segments$CHROM)

# Create a copy to modify
cell_df_updated <- cell_df

# Step 1: Set all CN_total==2 bins to "1|1" as baseline
cell_df_updated <- cell_df_updated %>%
  mutate(CN = ifelse(CN_total == 2, "1|1", CN))

new_segments_list <- list()

for (cell_name_curr in unique(loh_segments$cell_name)) {
  # Get new inferred CN-LOH segments for this cell
  loh_sub <- loh_segments %>% filter(cell_name == cell_name_curr)
  if (nrow(loh_sub) == 0) next
  
  message("Processing LOH for cell: ", cell_name_curr)
  
  # GRanges of CN-LOH
  loh_gr <- GRanges(
    seqnames = loh_sub$CHROM,
    ranges = IRanges(start = loh_sub$start, end = loh_sub$end)
  )
  
  # Subset cell bins for this cell with CN_total==2 and CN=="1|1" (baseline diploid bins)
  cell_sub <- cell_df_updated %>%
    filter(cell_name == cell_name_curr, CN_total == 2, CN == "1|1")
  
  if (nrow(cell_sub) == 0) next
  
  cell_gr <- GRanges(
    seqnames = cell_sub$CHROM,
    ranges = IRanges(start = cell_sub$START, end = cell_sub$END)
  )
  
  # Find overlaps between baseline diploid bins and LOH segments
  hits <- findOverlaps(cell_gr, loh_gr)
  if (length(hits) == 0) next
  
  cell_sub_indices <- which(cell_df_updated$cell_name == cell_name_curr & cell_df_updated$CN_total == 2 & cell_df_updated$CN == "1|1")
  bins_to_update <- cell_sub_indices[queryHits(hits)]
  
  # Update only those bins from "1|1" to "2|0"
  cell_df_updated[bins_to_update, "CN"] <- "2|0"
  # CN_total remains 2, no change needed
}


Processing LOH for cell: SMACS2T3VKE6

Processing LOH for cell: SMACS3WFHOU9

Processing LOH for cell: SMACS9H3UGVZ

Processing LOH for cell: SMACSBR61KY4

Processing LOH for cell: SMACSF5MYSSK

Processing LOH for cell: SMACSFUIX8H4

Processing LOH for cell: SMACSG1OVSEO

Processing LOH for cell: SMACSHIW5EMF

Processing LOH for cell: SMACSIGNW5HE

Processing LOH for cell: SMACSTLU27CI

Processing LOH for cell: SMACSW7REYP9

Processing LOH for cell: SMACSZJOM8X7



In [15]:
# Now rebuild seg_merged_df with updated cell_df_updated for CN_total==2 bins only

seg_merged_updated <- seg_merged_df

for (cell_name_curr in unique(cell_df_updated$cell_name)) {
    cell_sub <- cell_df_updated %>%
        filter(cell_name == cell_name_curr) %>%
        arrange(CHROM, START) %>%
        group_by(CHROM) %>%
        mutate(seg_id = data.table::rleid(CN)) %>%
        ungroup()
    
    # Now calculate seg_CN *per segment*
    cell_sub <- cell_sub %>%
        group_by(CHROM, seg_id) %>%
        mutate(seg_CN = CN[1]) %>%
        ungroup()


  
    if (nrow(cell_sub) == 0) next
    
    # Now filter post-segmentation
    new_segments <- cell_sub %>%
        group_by(CHROM, seg_id) %>%
        filter(seg_CN %in% c("1|1", "2|0")) %>%
        summarise(
        start = min(START),
        end = max(END),
        CN = seg_CN[1],
        CN_total = 2,
        chrom = CHROM[1],
        NBIN = n(),
        RDR_SUM = sum(RDR, na.rm = TRUE),
        RDR_MEAN = mean(RDR, na.rm = TRUE),
        VAF_MEAN = mean(BAF, na.rm = TRUE),
        VAF_ESTIMATE = ifelse(CN[1] == "2|0", 0, NA_real_),
        sample = sample[1],
        batch_name = batch_name[1],
        cell_type = cell_type[1],
        cell_name = cell_name[1],
        .groups = "drop"
    )
    new_segments$CHROM <- NA
    new_segments$seg_id <- NA
    # Remove old CN=2 segments for this cell
    seg_merged_updated <- seg_merged_updated %>%
    filter(!(cell_name == cell_name_curr & CN_total == 2))
    
    # Append new segments
    seg_merged_updated <- bind_rows(seg_merged_updated, new_segments)
    }
    
# Sort final segments
seg_merged_updated <- seg_merged_updated %>%
arrange(cell_name, chrom, start)


In [16]:
library(GenomicRanges)
library(dplyr)

# Subset relevant columns from original and updated
orig_sub <- cell_df %>%
  select(cell_name, CHROM, START, END, CN, CN_total)

updated_sub <- cell_df_updated %>%
  select(cell_name, CHROM, START, END, CN, CN_total)

# Create GRanges for original
orig_gr <- GRanges(
  seqnames = orig_sub$CHROM,
  ranges = IRanges(start = orig_sub$START, end = orig_sub$END),
  cell_name = orig_sub$cell_name,
  CN = orig_sub$CN,
  CN_total = orig_sub$CN_total
)

# Create GRanges for updated
updated_gr <- GRanges(
  seqnames = updated_sub$CHROM,
  ranges = IRanges(start = updated_sub$START, end = updated_sub$END),
  cell_name = updated_sub$cell_name,
  CN = updated_sub$CN,
  CN_total = updated_sub$CN_total
)

# Find overlaps (assuming bins match exactly by position and cell)
hits <- findOverlaps(orig_gr, updated_gr)

hit_df <- data.frame(
  orig_idx = queryHits(hits),
  updated_idx = subjectHits(hits)
)

# Keep only overlaps for same cell
hit_df <- hit_df %>%
  filter(mcols(orig_gr)$cell_name[orig_idx] == mcols(updated_gr)$cell_name[updated_idx])

# Add original and updated CN calls
hit_df <- hit_df %>%
  mutate(
    orig_CN = mcols(orig_gr)$CN[orig_idx],
    updated_CN = mcols(updated_gr)$CN[updated_idx],
    orig_CN_total = mcols(orig_gr)$CN_total[orig_idx],
    updated_CN_total = mcols(updated_gr)$CN_total[updated_idx],
    changed = (orig_CN != updated_CN) | (orig_CN_total != updated_CN_total)
  )

# Show bins where CN or CN_total changed
changed_bins <- hit_df %>% filter(changed)

print(changed_bins)


     orig_idx updated_idx orig_CN updated_CN orig_CN_total updated_CN_total
1        3807        3807     2|0        1|1             2                2
2        3808        3808     2|0        1|1             2                2
3        3809        3809     2|0        1|1             2                2
4        3810        3810     2|0        1|1             2                2
5        3811        3811     2|0        1|1             2                2
6        3812        3812     2|0        1|1             2                2
7        3813        3813     2|0        1|1             2                2
8        4795        4795     2|0        1|1             2                2
9        4796        4796     2|0        1|1             2                2
10       4797        4797     2|0        1|1             2                2
11       4798        4798     2|0        1|1             2                2
12       4799        4799     2|0        1|1             2                2
13       480

In [17]:
unique(changed_bins$orig_CN)

[1] "2|0" "1|1"

In [18]:
unique(changed_bins$updated_CN)

[1] "1|1" "2|0"

In [19]:
names(seg_merged_updated)

[1] "start"        "end"          "CN"           "chrom"        "NBIN"        
 [6] "RDR_SUM"      "RDR_MEAN"     "VAF_MEAN"     "VAF_ESTIMATE" "sample"      
[11] "cell_name"    "batch_name"   "cell_type"    "CN_A"         "CN_B"        
[16] "CN_total"     "length"       "amt_dna"      "is_CNV"       "is_LOH"      
[21] "CHROM"        "seg_id"

In [20]:
write_rds(seg_merged_updated[,orig_seg_merged_df_colnames], "seg_merged_df_CN-LOH-fix.rds")

In [21]:
write_rds(cell_df_updated[,orig_cell_df_colnames], "cell_df_CN-LOH-fix.rds")